In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # 更改預設gpu為1
import torch
print(
    # 確認 torch 的版本
    f'PyTorch version {torch.__version__}\n' +
    # 確認是否有 GPU 裝置
    f'GPU-enabled installation? {torch.cuda.is_available()}\n'
    # 確認GPU 0裝置名字
    f'GPU-device name? {torch.cuda.get_device_name(0)}\n'
    # 確認GPU 1裝置名字
    #f'GPU-device name? {torch.cuda.get_device_name(1)}\n'
)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to specified size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

# Specify directory paths
data_dir = "./Data"  # Replace with your directory path
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(128, num_classes)  # Set num_classes based on your problem

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.flatten(x)
        x = self.relu4(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

In [6]:
num_epochs = 10  # Adjust number of epochs

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.5650
Epoch [2/10], Loss: 0.0195
Epoch [3/10], Loss: 0.0299
Epoch [4/10], Loss: 0.0071
Epoch [5/10], Loss: 0.0008
Epoch [6/10], Loss: 0.0018
Epoch [7/10], Loss: 0.0001
Epoch [8/10], Loss: 0.0004
Epoch [9/10], Loss: 0.0002
Epoch [10/10], Loss: 0.0004


In [7]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 100.00%


In [8]:
torch.save(model.state_dict(), "NewData/savemodel100.pth")

In [19]:
import torch
from torchvision import transforms
from PIL import Image

# Define the path to your model file
model_path = 'Data/savemodel100.pth'

# Load the model
model = CNNModel()  # Replace with the architecture of your model
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to the training size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

# Load and preprocess the input image
input_image_path = 'Data/test/9X/image_9X_301.jpg'
input_image = Image.open(input_image_path).convert("RGB")
input_tensor = transform(input_image).unsqueeze(0)  # Add batch dimension

# Perform inference
with torch.no_grad():
    output = model(input_tensor)

# Assuming output is a tensor, you can convert it to probabilities or predicted class
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

# Print the result
print(f"Predicted class: {predicted_class}, Probability: {probabilities[predicted_class]:.4f}")

Predicted class: 5, Probability: 0.9997
